<a href="https://colab.research.google.com/github/nehaal10/Cloud_deployment/blob/main/DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install remotezip tqdm opencv-python einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 2.6 MB/s eta 0:00:00
  Created wheel for remotezip: filename=remotezip-0.12.1-py3-none-any.whl size=7949 sha256=2d98a7929445f7134a00561dde16edb10c3a1f9ca3238103926fcf406a4039e7
  Stored in directory: /root/.cache/pip/wheels/60/74/6c/b12b4c8fb4b7ab08f495ce17e88f1e98835268af7a8ad5588f
Successfully built remotezip


In [2]:
!pip install remotezip tqdm opencv-python
!pip install -q git+https://github.com/tensorflow/docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done


In [3]:
!pip install remotezip tqdm opencv-python==4.5.2.52 opencv-python-headless==4.5.2.52 tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 32.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-non

In [4]:
import os 
import shutil
from zipfile import ZipFile
import cv2
import io
import numpy as np
from PIL import Image
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder
import pickle 
import einops
import matplotlib.pyplot as plt
import imageio
from tensorflow_docs.vis import embed
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

In [5]:
import gc
gc.collect()

0

In [ ]:
os.chdir('/content/drive/MyDrive/kaggle')
path=os.getcwd()
os.environ['KAGGLE_CONFIG_DIR']=path
!kaggle datasets download -d alirakhmaev/ucf-crime-full 

 87% 21.2G/24.4G [03:01<00:20, 167MB/s]

In [ ]:
HIGHT=224
WIDTH=224

In [ ]:
def format_frames(frame, output_size=(HIGHT,WIDTH)):
  """
    Pad and resize an image from a video.
    
    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame=frame/255.0
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def delete(p):
  li=p.split('/')
  li.pop()
  p='/'.join(li)
  shutil.rmtree(p)

def encode_target(class_name):
  lb=LabelEncoder()
  return lb.fit_transform(class_name),lb
   

def extract(p):
  with  ZipFile('/content/drive/MyDrive/kaggle/ucf-crime-full.zip') as f:
    f.extract(p,'/content')



def get_call_path(list_of_vedio,path):
  paths=[]
  cname=[]
  veds=[]
  #---
  #/content/drive/MyDrive/kaggle/Abuse/Abuse001_x264.mp4

  for i in veds:
    class_name= i.split('/')[0]
    path_to_save=path+'/{}/{}'.format(class_name,i.split('/')[-1])
    paths.append(path_to_save)
    cname.append(class_name)
  cname,lb=encode_target(cname)

  return paths,cname,list_of_vedio,lb

# def get_frames(p,orgi_path,n_frame):
#     extract(orgi_path)

#     cap = cv2.VideoCapture(p)
#     frames = []
#     main=[]
#     i=0
#     k=n_frame
#     len=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     for i in range(len): 
#       ret, frame = cap.read()
#       if i==k:
#         main.append(np.array(frames))
#         k=k*2
#       else:
#         frame=format_frames(frame)
#         frames.append(frame)

#     delete(p)

#     return main

In [32]:
def frames_from_video_file(p, orgi_path,n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  extract(orgi_path)
  result = []
  src = cv2.VideoCapture(str(p))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]
  delete(p)
  return result

In [33]:
class FrameGenerator:
  def __init__(self,orgi_vedios,n_frames,training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """

    self.frames=n_frames
    self.orgi=orgi_vedios
    self.training = training
    #self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    #self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

    
  def __call__(self):
    #if self.training:
     # random.shuffle(pairs)
    
    paths,class_name,list_of_vedio,lb=get_call_path(self.orgi,'/content') 
    pairs=zip(paths,list_of_vedio,class_name)
    for p,o,c in pairs:
      video_frames = frames_from_video_file(p,o,self.frames) 
      yield video_frames, c 

In [40]:
import random

with ZipFile('/content/drive/MyDrive/kaggle/ucf-crime-full.zip') as f:
  list_of_vedio=f.namelist()

random.shuffle(list_of_vedio)

In [42]:
list_of_vedio

['Shooting/Shooting024_x264.mp4',
 'Assault/Assault004_x264.mp4',
 'Arrest/Arrest011_x264.mp4',
 'Fighting/Fighting007_x264.mp4',
 'Abuse/Abuse046_x264.mp4',
 'RoadAccidents/RoadAccidents110_x264.mp4',
 'Burglary/Burglary029_x264.mp4',
 'Stealing/Stealing065_x264.mp4',
 'Shoplifting/Shoplifting026_x264.mp4',
 'RoadAccidents/RoadAccidents108_x264.mp4',
 'Robbery/Robbery015_x264.mp4',
 'Explosion/Explosion024_x264.mp4',
 'Arrest/Arrest017_x264.mp4',
 'Fighting/Fighting033_x264.mp4',
 'Stealing/Stealing080_x264.mp4',
 'Normal_Videos_for_Event_Recognition/Normal_Videos_828_x264.mp4',
 'Shoplifting/Shoplifting001_x264.mp4',
 'Stealing/Stealing003_x264.mp4',
 'Stealing/Stealing107_x264.mp4',
 'Assault/Assault052_x264.mp4',
 'Vandalism/Vandalism025_x264.mp4',
 'Burglary/Burglary034_x264.mp4',
 'Shooting/Shooting046_x264.mp4',
 'Robbery/Robbery143_x264.mp4',
 'RoadAccidents/RoadAccidents007_x264.mp4',
 'Shooting/Shooting018_x264.mp4',
 'Burglary/Burglary075_x264.mp4',
 'Robbery/Robbery122_x264

In [18]:

with ZipFile('/content/drive/MyDrive/kaggle/ucf-crime-full.zip') as f:
  list_of_vedio=f.namelist()

veds=[]
for i in list_of_vedio:
  if i.split('/')[0]=='Abuse' or i.split('/')[0]=='Normal_Videos_for_Event_Recognition' or i.split('/')[0]=='Assault':
      veds.append(i)


random.shuffle(veds)
train=int(len(veds)*0.8)
val=int(len(veds)*0.15)
test=train+val
train_list=veds[:train]
val_list=veds[train:test]
test_list=veds[test:]


output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_list,32,training=True),
                                          output_signature = output_signature)

val_ds=tf.data.Dataset.from_generator(FrameGenerator(val_list,32,training=True),
                                          output_signature = output_signature)

test_ds=tf.data.Dataset.from_generator(FrameGenerator(test_list,32,training=True),
                                          output_signature = output_signature)


In [19]:
for i , j in train_ds.take(1):
  print(i.shape)

In [59]:
from tensorflow._api.v2.data import AUTOTUNE
train_ds=train_ds.shuffle(500).batch(4).cache().prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.shuffle(500).batch(4).cache().prefetch(buffer_size=AUTOTUNE)
test_ds=test_ds.shuffle(500).batch(4).cache().prefetch(buffer_size=AUTOTUNE)

In [60]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


movinet_a0_base/
movinet_a0_base/checkpoint
movinet_a0_base/ckpt-1.data-00000-of-00001
movinet_a0_base/ckpt-1.index


In [61]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [62]:
model = build_classifier(4, 32, 224, backbone, 2)

In [63]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

In [64]:
checkpoint_path = "/content/drive/MyDrive/checkpoint/1st model/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True
                                                 )

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [65]:
!nvidia-smi

Sat Mar 25 07:56:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |   2385MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
results = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=100,
                    callbacks=[cp_callback,callback])

Epoch 1/100
